## A notebook for fine-tune llama 2 and verify fine-tune results

Can be running on colab or Linux systems

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
# login to huggingface to download llama 2, or skip this to load model from antoher resource
from huggingface_hub import login
login()

In [ ]:
# pre defined
dataset_name = "databricks/databricks-dolly-15k"
model_name = "meta-llama/Llama-2-7b-chat-hf"
new_model_name = "Llama-2-7b-dolly-15k"
max_seq_length = 2048 # minimal length, roughly 2048*3/4 words

In [ ]:
# Load dataset 
dataset = load_dataset(dataset_name, split="train")

# Load base model, model can be loaded in 4bit and nested quant with bitsandbytes 
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
args = TrainingArguments(
    output_dir=new_model_name,
    num_train_epochs=3,
    per_device_train_batch_size=4
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
)

# format instruction
def formatting_func(example):
    '''
    The goal of this fine tune is to take in the question and groundtruth as context,
    and produce an output that can be based on the given context.
    '''
    text = f"""### context:
            {example['category']}{example['context']}

            ### Input:
            {example['instruction']}

            ### Response:
            {example['response']}
            """
    return text

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=formatting_func,
    args=args,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model_name)

In [ ]:
# Load the TensorBoard notebook extension
# %load_ext tensorboard
# %tensorboard --logdir results/runs